In [ ]:
from collections import defaultdict
import json
import os
import numpy as np
import cv2
from research.utils.data_access_utils import RDSAccessUtils
from research_lib.utils.data_access_utils import S3AccessUtils
from weight_estimation.body_parts import core_body_parts
from weight_estimation.utils import convert_to_world_point_arr, get_left_right_keypoint_arrs, normalize_left_right_keypoint_arrs, CameraMetadata, \
    stabilize_keypoints, convert_to_nn_input
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
import torch
from research.weight_estimation.keypoint_utils.optics import pixel2world
import pandas as pd


<h1> Load in toy fish annotations </h1>

In [ ]:
os.environ['PLALI_SQL_CREDENTIALS'] = '/run/secrets/plali_sql_credentials'
rds = RDSAccessUtils(json.load(open(os.environ['PLALI_SQL_CREDENTIALS'])))

def get_annotated_data(workflow_id, metadata_type):
    query = """
        select * from plali.plali_annotations x
        inner join 
        ( select a.id as plali_image_id, a.images, a.metadata, b.id as workflow_id, b.name from plali.plali_images a
        inner join plali.plali_workflows b
        on a.workflow_id = b.id ) y
        on x.plali_image_id = y.plali_image_id
        where workflow_id = '{}';
    """.format(workflow_id)

    annotated_df = rds.extract_from_database(query)
    annotated_df = annotated_df[annotated_df.metadata.apply(lambda x: str(x.get('type')) == metadata_type)]
    return annotated_df



class AnnotationFormatError(Exception):
    pass


def add_anns(annotated_df):
    anns = []
    for idx, row in annotated_df.iterrows():
        try:
            raw_ann = row.annotation
            if 'skipReasons' in raw_ann:
                raise AnnotationFormatError

            ann = {'leftCrop': [], 'rightCrop': []}

            for side in ['leftCrop', 'rightCrop']:
                for raw_item in row.annotation[side]['annotation']['annotations']:
                    if 'xCrop' not in raw_item or 'yCrop' not in raw_item:
                        raise AnnotationFormatError
                    item = {
                        'xCrop': raw_item['xCrop'],
                        'yCrop': raw_item['yCrop'],
                        'xFrame': raw_item['xCrop'],
                        'yFrame': raw_item['yCrop'],
                        'keypointType': raw_item['category']
                    }

                    ann[side].append(item)

            if any([len(ann[side]) != 11 for side in ['leftCrop', 'rightCrop']]):
                raise AnnotationFormatError

            anns.append(ann)

        except AnnotationFormatError as err:
            anns.append(None)

    annotated_df['ann'] = anns



In [ ]:
import json
from typing import Dict
import cv2
import numpy as np


IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000


def get_camera_parameters(params: Dict) -> Dict:
    """Return individual camera parameters from JSON stereo parameters contents."""

    camera_matrix_1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    camera_matrix_2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    dist_coeffs_1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                    params['CameraParameters1']['TangentialDistortion'] + \
                    [params['CameraParameters1']['RadialDistortion'][2]]
    dist_coeffs_1 = np.array(dist_coeffs_1)

    dist_coeffs_2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                    params['CameraParameters2']['TangentialDistortion'] + \
                    [params['CameraParameters2']['RadialDistortion'][2]]
    dist_coeffs_2 = np.array(dist_coeffs_2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(camera_matrix_1, dist_coeffs_1,
                                                               camera_matrix_2,
                                                               dist_coeffs_2, image_size, R, T,
                                                               None,
                                                               None,
                                                               None, None, None,
                                                               cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(camera_matrix_1, dist_coeffs_1, R1, P1, image_size,
                                            cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(camera_matrix_2, dist_coeffs_2, R2, P2, image_size,
                                             cv2.CV_16SC2)

    params = {
        'left_maps': left_maps,
        'right_maps': right_maps,
        'camera_matrix_1': camera_matrix_1,
        'dist_coeffs_1': dist_coeffs_1,
        'R1': R1,
        'P1': P1,
        'camera_matrix_2': camera_matrix_2,
        'dist_coeffs_2': dist_coeffs_2,
        'R2': R2,
        'P2': P2
    }
    return params


def unrectify(ann: Dict, params: Dict):
    """Un-rectify ann with params."""

    left_maps = params['left_maps']
    right_maps = params['right_maps']

    ann_u = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = left_maps[0][y, x]
            elif side == 'rightCrop':
                x_new, y_new = right_maps[0][y, x]
            else:
                raise Exception('Invalid side!')

            ann_u[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_u



def rectify(ann: Dict, params: Dict) -> Dict:
    """Rectify ann with params."""

    camera_matrix_1 = params['camera_matrix_1']
    dist_coeffs_1 = params['dist_coeffs_1']
    R1 = params['R1']
    P1 = params['P1']

    camera_matrix_2 = params['camera_matrix_2']
    dist_coeffs_2 = params['dist_coeffs_2']
    R2 = params['R2']
    P2 = params['P2']

    ann_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[x, y]]).astype(float),
                        camera_matrix_1,
                        dist_coeffs_1,
                        R=R1,
                        P=P1)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[x, y]]).astype(float),
                        camera_matrix_2,
                        dist_coeffs_2,
                        R=R2,
                        P=P2)[0][0]
            else:
                raise Exception('Invalid side!')

            ann_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_r


def un_re_rectify(df, params_o, params_n):

    ann_u_rs = []
    for idx, row in df.iterrows():
        ann = row.ann
        if ann is None:
            ann_u_rs.append(None)
            continue

        # un-rectify with original params
        ann_u = unrectify(ann, params_o)
        if ann_u:

            # re-rectify with new params
            ann_u_r = rectify(ann_u, params_n)
            ann_u_rs.append(ann_u_r)
        else:
            ann_u_rs.append(None)

    df['ann_u_r'] = ann_u_rs


def add_camera_metadata(df, stereo_parameters):
    
    camera_metadata = {
        'focalLengthPixel': stereo_parameters['CameraParameters1']['FocalLength'][0],
        'baseline': abs(stereo_parameters['TranslationOfCamera2'][0] / 1e3),
        'focalLength': stereo_parameters['CameraParameters1']['FocalLength'][0] * 3.45e-6,
        'pixelCountWidth': 4096,
        'pixelCountHeight': 3000,
        'imageSensorWidth': 0.01412,
        'imageSensorHeight': 0.01035
    }

    df['camera_metadata'] = json.dumps(camera_metadata)
    

def add_weights(df, k):
    
    weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    pred_weights = []
    for idx, row in df.iterrows():
        ann = row.ann_u_r
        camera_metadata = json.loads(row.camera_metadata)
        if ann is not None:
            cm = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, _, _ = weight_estimator.predict(ann, cm)
            pred_weights.append(weight)
        else:
            pred_weights.append(None)
    
    df['weight_{}'.format(k)] = pred_weights

In [ ]:
experiment = {
    'name': 'B2_fish_moving_around_v1',
    'workflow_id': '00000000-0000-0000-0000-000000000055',
    'metadata_type': 'Dale P3 post-swap enclosure'
}

In [ ]:
name = experiment['name']
df = get_annotated_data(experiment['workflow_id'], experiment['metadata_type'])
add_anns(df)


In [ ]:
s3 = S3AccessUtils('/root/data')

stereo_parameter_urls = [
    ('parameters1', 's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-01-07T12:17:15.547471000Z_L40076048_R40076044_stereo-parameters.json'),
    ('parameters2', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-10T13:57:48Z-pfe-1421920048928-187-4bd8/cal_output/2021-03-10T14-07-03.821272000Z/stereo_params.json'),
    ('rot1', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A17%3A33Z-pfe-1421920048928-9f6-46e1/cal_output/2021-04-06T10-32-38.492361000Z/stereo_params.json'),
    ('rot2', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A25%3A01Z-pfe-1421920048928-1b8-4398/cal_output/2021-04-06T10-59-45.370389000Z/stereo_params.json'),
    ('rot3', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A32%3A30Z-pfe-1421920048928-5a9-46c6/cal_output/2021-04-06T10-45-35.798195000Z/stereo_params.json'),
    ('rot4', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A40%3A44Z-pfe-1421920048928-ea1-4697/cal_output/2021-04-06T11-27-09.267188000Z/stereo_params.json'),
    ('rot5', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A48%3A17Z-pfe-1421920048928-dcb-445c/cal_output/2021-04-06T11-15-19.086423000Z/stereo_params.json'),
    ('shake1', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A57%3A54Z-pfe-1421920048928-675-46e5/cal_output/2021-04-06T11-48-27.629782000Z/stereo_params.json'),
    ('shake2', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T11%3A06%3A52Z-pfe-1421920048928-5db-4c60/cal_output/2021-04-06T12-00-13.274660000Z/stereo_params.json'),
    ('shake3', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T11%3A41%3A00Z-pfe-1421920048928-f86-48bb/cal_output/2021-04-06T12-11-21.222269000Z/stereo_params.json'),
    ('shake4', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T11%3A50%3A04Z-pfe-1421920048928-635-49bf/cal_output/2021-04-06T12-36-00.638934000Z/stereo_params.json'),
    ('shake5', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T12%3A06%3A14Z-pfe-1421920048928-390-424d/cal_output/2021-04-06T12-22-49.207767000Z/stereo_params.json')
]

stereo_parameter_urls = [(x, y.replace('%3A', ':')) for x, y in stereo_parameter_urls]
stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameter_urls[1][1])
stereo_parameters = json.load(open(stereo_parameters_f))
params_o = get_camera_parameters(stereo_parameters)


for k, stereo_parameter_url in stereo_parameter_urls:
    if stereo_parameter_url.startswith('s3://'):
        stereo_parameters_components = stereo_parameter_url.replace('s3://', '').split('/')
        bucket = stereo_parameters_components[0]
        key = os.path.join(*stereo_parameters_components[1:])
        stereo_parameters_f = s3.download_from_s3(bucket, key)
    elif stereo_parameter_url.startswith('https://'):
        stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameter_url)
    else:
        raise Exception('Unrecognized URL format')
    stereo_parameters = json.load(open(stereo_parameters_f))
    params = get_camera_parameters(stereo_parameters)
    un_re_rectify(df, params_o, params)
    add_camera_metadata(df, stereo_parameters)
    add_weights(df, k)
    print(k, df['weight_{}'.format(k)].mean())
    




In [ ]:
from matplotlib import pyplot as plt

In [ ]:
errors = df['weight_rot1'].values - df['weight_rot3'].values

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(errors)
plt.grid()
plt.show()

In [ ]:
def construct_new_stereo_parameters(stereo_parameters_o, stereo_parameters_n, 
                                    change_PP_1=False, change_PP_2=False, 
                                    change_focal_length_1=False, change_focal_length_2=False,
                                    change_rd_1=False, change_rd_2=False, change_td_1=False, change_td_2=False, 
                                    change_R=False, change_T=False):
    
    stereo_parameters_new = dict(stereo_parameters_o)
    
    if change_PP_1:
        stereo_parameters_new['CameraParameters1']['PrincipalPoint'] = \
            stereo_parameters_n['CameraParameters1']['PrincipalPoint']
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][2][:2] = \
            stereo_parameters_n['CameraParameters1']['PrincipalPoint']
    if change_PP_2:
        stereo_parameters_new['CameraParameters2']['PrincipalPoint'] = \
            stereo_parameters_n['CameraParameters2']['PrincipalPoint']
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][2][:2] = \
            stereo_parameters_n['CameraParameters2']['PrincipalPoint']
    if change_focal_length_1:
        stereo_parameters_new['CameraParameters1']['FocalLength'] = \
            stereo_parameters_n['CameraParameters1']['FocalLength']
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][0][0] = \
            stereo_parameters_n['CameraParameters1']['FocalLength'][0]
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][1][1] = \
            stereo_parameters_n['CameraParameters1']['FocalLength'][1]
    if change_focal_length_2:
        stereo_parameters_new['CameraParameters2']['FocalLength'] = \
            stereo_parameters_n['CameraParameters2']['FocalLength']
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][0][0] = \
            stereo_parameters_n['CameraParameters2']['FocalLength'][0]
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][1][1] = \
            stereo_parameters_n['CameraParameters2']['FocalLength'][1]
    if change_rd_1:
        stereo_parameters_new['CameraParameters1']['RadialDistortion'] = \
            stereo_parameters_n['CameraParameters1']['RadialDistortion']
    if change_rd_2:
        stereo_parameters_new['CameraParameters2']['RadialDistortion'] = \
            stereo_parameters_n['CameraParameters2']['RadialDistortion']
    if change_td_1:
        stereo_parameters_new['CameraParameters1']['TangentialDistortion'] = \
            stereo_parameters_n['CameraParameters1']['TangentialDistortion']
    if change_td_2:
        stereo_parameters_new['CameraParameters2']['TangentialDistortion'] = \
            stereo_parameters_n['CameraParameters2']['TangentialDistortion']
    if change_R:
        stereo_parameters_new['RotationOfCamera2'] = stereo_parameters_n['RotationOfCamera2']
    if change_T:
        stereo_parameters_new['TranslationOfCamera2'] = stereo_parameters_n['TranslationOfCamera2']
        
    return stereo_parameters_new

In [ ]:
def download_from_url(stereo_parameter_url):
    if stereo_parameter_url.startswith('s3://'):
        stereo_parameters_components = stereo_parameter_url.replace('s3://', '').split('/')
        bucket = stereo_parameters_components[0]
        key = os.path.join(*stereo_parameters_components[1:])
        stereo_parameters_f = s3.download_from_s3(bucket, key)
    elif stereo_parameter_url.startswith('https://'):
        stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameter_url)
    else:
        raise Exception('Unrecognized URL format')
    return stereo_parameters_f

stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameter_urls[1][1])
stereo_parameters = json.load(open(stereo_parameters_f))
params_orig = get_camera_parameters(stereo_parameters)

analysis_data = defaultdict(list)
parameter_names = ['PP_1', 'PP_2', 'F1', 'F2', 'RD1', 'RD2', 'TD1', 'TD2', 'R', 'T']

for idx in range(10):
    analysis_data['name'].append(parameter_names[idx])
    for k in range(len(stereo_parameter_urls)):
        bool_array = [False] * 10
        bool_array[idx] = True

        stereo_parameters_o_f = download_from_url(stereo_parameter_urls[0][1])
        stereo_parameters_n_f = download_from_url(stereo_parameter_urls[k][1])
        stereo_params_o = json.load(open(stereo_parameters_o_f))
        stereo_params_new = json.load(open(stereo_parameters_n_f))

        params_o = get_camera_parameters(stereo_params_o)
        stereo_params_n = construct_new_stereo_parameters(stereo_params_o, 
                                                          stereo_params_new, 
                                                          change_PP_1=bool_array[0],
                                                          change_PP_2=bool_array[1],
                                                          change_focal_length_1=bool_array[2],
                                                          change_focal_length_2=bool_array[3],
                                                          change_rd_1=bool_array[4],
                                                          change_rd_2=bool_array[5],
                                                          change_td_1=bool_array[6],
                                                          change_td_2=bool_array[7],
                                                          change_R=bool_array[8],
                                                          change_T=bool_array[9]
                                                          )

        params_n = get_camera_parameters(stereo_params_n)
        un_re_rectify(df, params_orig, params_n)
        add_camera_metadata(df, stereo_params_n)
        add_weights(df)

        analysis_data['s{}_prime_value'.format(k)].append((df.weight.mean() - 3109.229432855859) / 3109.229432855859)



In [ ]:
pd.DataFrame(analysis_data)

In [ ]:
data = json.load(open('/root/data/alok/biomass_estimation/playground/depth_akpd.json'))

In [ ]:
df = pd.read_csv('/root/data/alok/biomass_estimation/playground/depthable.csv')

In [ ]:
df.head()

In [ ]:
len(data)

In [ ]:
df['ann'] = data

In [ ]:
def add_weights(df):
    
    weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    pred_weights = []
    for idx, row in df.iterrows():
        ann = row.ann
        camera_metadata = json.loads(row.camera_metadata.replace("'", '"'))
        if ann is not None:
            cm = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, _, _ = weight_estimator.predict(ann, cm)
            pred_weights.append(weight)
        else:
            pred_weights.append(None)
    
    df['weight'] = pred_weights

In [ ]:
add_weights(df)